# FPA-Net: Frequency-Guided Position-Based Attention Network for Land Cover Image Segmentation

## Author: **<small><small>  AL SHAHRIAR RUBEL </small></small>**



# Citation
Please **cite** the following research article if you use any part of this work <br><br>
***Rubel, Al Shahriar, and Frank Y. Shih. "FPA-Net: Frequency-guided Position-based Attention Network for Land Cover Image Segmentation." International Journal of Pattern Recognition and Artificial Intelligence (2023).***

### Dataset Preparation

In [ ]:
# Connecting Google Drive with Google Colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pytorch-lightning
!pip install segmentation-models-pytorch
!pip install -U albumentations
!pip install -q torchinfo
!pip install pandas
!pip install matplotlib

In [ ]:
import os
import cv2
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pytorch_lightning as pl
from torch import nn
from tqdm.notebook import tqdm

In [ ]:
pl.seed_everything(42, workers=True)

Extract Data

In [ ]:
IMAGE_SIZE = 320
BATCH_SIZE = 16
EPOCHS = 100

color_dict = pd.read_csv('drive/MyDrive/MLProject/data/class_dict.csv')
CLASSES = color_dict['name']
print(color_dict)

In [ ]:
from glob import glob
from sklearn.utils import shuffle

pd_dataset = pd.DataFrame({'IMAGES': sorted(glob("drive/MyDrive/MLProject/data/train/*_sat.jpg")), 'MASKS': sorted(glob("drive/MyDrive/MLProject/data/train/*_mask.png"))
})
pd_dataset = shuffle(pd_dataset)
pd_dataset.reset_index(inplace=True, drop=True)
pd_dataset.head()


In [ ]:
from sklearn.model_selection import train_test_split

pd_train, pd_test = train_test_split(pd_dataset, test_size=0.25, random_state=0)
pd_train, pd_val = train_test_split(pd_train, test_size=0.2, random_state=0)

print("Training set size:", len(pd_train))
print("Validation set size:", len(pd_val))
print("Testing set size:", len(pd_test))

In [ ]:
index = 200

sample_img = cv2.imread(pd_train.iloc[index].IMAGES)
sample_img = cv2.cvtColor(sample_img, cv2.COLOR_BGR2RGB)

sample_msk = cv2.imread(pd_train.iloc[index].MASKS)
sample_msk = cv2.cvtColor(sample_msk, cv2.COLOR_BGR2RGB)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 10))

ax1.set_title('IMAGE')
ax1.imshow(sample_img)

ax2.set_title('MASK')
ax2.imshow(sample_msk)

utility functions

In [ ]:
def rgb2category(rgb_mask):
    category_mask = np.zeros(rgb_mask.shape[:2], dtype=np.int8)
    for i, row in color_dict.iterrows():
        category_mask += (np.all(rgb_mask.reshape((-1, 3)) == (row['r'], row['g'], row['b']), axis=1).reshape(rgb_mask.shape[:2]) * i)
    return category_mask

def category2rgb(category_mask):
    rgb_mask = np.zeros(category_mask.shape[:2] + (3,))
    for i, row in color_dict.iterrows():
        rgb_mask[category_mask==i] = (row['r'], row['g'], row['b'])
    return np.uint8(rgb_mask)

Data Augmentations & Transformations

In [ ]:
import albumentations as aug

train_augment = aug.Compose([
    aug.Resize(IMAGE_SIZE, IMAGE_SIZE),
    aug.HorizontalFlip(p=0.5),
    aug.VerticalFlip(p=0.5),
    aug.RandomBrightnessContrast(p=0.3)
])

test_augment = aug.Compose([
    aug.Resize(IMAGE_SIZE, IMAGE_SIZE),
    aug.RandomBrightnessContrast(p=0.3)
])

Create PyTorch Dataset

In [ ]:
from torch.utils.data import Dataset, DataLoader

class SegmentationDataset(Dataset):
    def __init__(self, df, augmentations=None):
        self.df = df
        self.augmentations = augmentations

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index: int):
        row = self.df.iloc[index]

        image = cv2.imread(row.IMAGES)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        mask = cv2.imread(row.MASKS)
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)

        if self.augmentations:
            data = self.augmentations(image=image, mask=mask)
            image = data['image']
            mask = data['mask']

        mask = rgb2category(mask)

        image = np.transpose(image, (2, 0, 1)).astype(np.float64)
        mask = np.expand_dims(mask, axis=0)

        image = torch.Tensor(image) / 255.0
        mask = torch.Tensor(mask).long()

        return image, mask

In [ ]:
class SegmentationDataModule(pl.LightningDataModule):
    def __init__(self, pd_train, pd_val, pd_test, batch_size=10):
        super().__init__()
        self.pd_train = pd_train
        self.pd_val = pd_val
        self.pd_test = pd_test
        self.batch_size=batch_size

    def setup(self, stage=None):
        self.train_dataset = SegmentationDataset(self.pd_train, train_augment)
        self.val_dataset = SegmentationDataset(self.pd_val, test_augment)
        self.test_dataset = SegmentationDataset(self.pd_test, test_augment)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=2)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size // 2, shuffle=False, num_workers=1)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size // 2, shuffle=False, num_workers=1)

In [ ]:
data_module = SegmentationDataModule(pd_train, pd_val, pd_test, batch_size=BATCH_SIZE)
data_module.setup()

In [ ]:
image, mask = next(iter(data_module.train_dataloader()))
image.shape, mask.shape

### Model Preparation (Before Training)

In [ ]:
!pip install torch-dct

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_dct as dct

from segmentation_models_pytorch.base import modules as md

class FrequencyFeature(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, kernel_size=3):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding = 'same')
    def forward(self, im):
        f_threshold = 10
        filtermask = torch.ones(im.shape[2],im.shape[3]).to(DEVICE)
        for i in range(f_threshold+1):
          for j in range(f_threshold+1):
            if j<=f_threshold - i:
              filtermask[i,j] = 0
        x = dct.dct_2d(im)
        x = x * filtermask

        x = dct.idct_2d(x)

        return x

class PAB(nn.Module):
    def __init__(self, in_channels, out_channels, pab_channels=64):
        super(PAB, self).__init__()
        self.pab_channels = pab_channels
        self.in_channels = in_channels
        self.top_conv = nn.Conv2d(in_channels, pab_channels, kernel_size=1)
        self.center_conv = nn.Conv2d(in_channels, pab_channels, kernel_size=1)
        self.bottom_conv = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1)
        self.map_softmax = nn.Softmax(dim=1)
        self.out_conv = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1)

    def forward(self, x):
        bsize = x.size()[0]
        h = x.size()[2]
        w = x.size()[3]
        x_top = self.top_conv(x)
        x_center = self.center_conv(x)
        x_bottom = self.bottom_conv(x)

        x_top = x_top.flatten(2)
        x_center = x_center.flatten(2).transpose(1, 2)
        x_bottom = x_bottom.flatten(2).transpose(1, 2)

        sp_map = torch.matmul(x_center, x_top)
        sp_map = self.map_softmax(sp_map.view(bsize, -1)).view(bsize, h * w, h * w)
        sp_map = torch.matmul(sp_map, x_bottom)
        sp_map = sp_map.reshape(bsize, self.in_channels, h, w)
        x = x + sp_map
        x = self.out_conv(x)
        return x

class DecoderBlock(nn.Module):
    def __init__(self, in_channels, skip_channels, out_channels, use_batchnorm=True):
        super().__init__()
        self.conv1 = md.Conv2dReLU(
            in_channels + skip_channels,
            out_channels,
            kernel_size=3,
            padding=1,
            use_batchnorm=use_batchnorm,
        )
        self.conv2 = md.Conv2dReLU(
            out_channels,
            out_channels,
            kernel_size=3,
            padding=1,
            use_batchnorm=use_batchnorm,
        )

    def forward(self, x, skip=None):
        x = F.interpolate(x, scale_factor=2, mode="nearest")
        if skip is not None:
            x = torch.cat([x, skip], dim=1)
        x = self.conv1(x)
        x = self.conv2(x)
        return x


class FPAnetDecoder(nn.Module):
    def __init__(
        self,
        encoder_channels,
        decoder_channels,
        n_blocks=5,
        reduction=16,
        use_batchnorm=True,
        pab_channels=64,
    ):
        super().__init__()

        if n_blocks != len(decoder_channels):
            raise ValueError(
                "Model depth is {}, but you provide `decoder_channels` for {} blocks.".format(
                    n_blocks, len(decoder_channels)
                )
            )

        # remove first skip with same spatial resolution
        encoder_channels = encoder_channels[1:]

        # reverse channels to start from head of encoder
        encoder_channels = encoder_channels[::-1]

        # computing blocks input and output channels
        head_channels = encoder_channels[0]
        in_channels = [head_channels] + list(decoder_channels[:-1])
        skip_channels = list(encoder_channels[1:]) + [0]
        out_channels = decoder_channels

        self.center = PAB(head_channels, head_channels, pab_channels=pab_channels)

        # combine decoder keyword arguments
        kwargs = dict(use_batchnorm=use_batchnorm)  # no attention type here

        blocks = [
            DecoderBlock(in_ch, skip_ch, out_ch, **kwargs)
            for in_ch, skip_ch, out_ch in zip(in_channels, skip_channels, out_channels)
        ]
        # for the last we dont have skip connection -> use simple decoder block
        self.blocks = nn.ModuleList(blocks)

    def forward(self, *features):

        features = features[1:]  # remove first skip with same spatial resolution
        features = features[::-1]  # reverse channels to start from head of encoder

        head = features[0]
        skips = features[1:]

        x = self.center(head)

        for i, decoder_block in enumerate(self.blocks):
            skip = skips[i] if i < len(skips) else None
            x = decoder_block(x, skip)

        return x

In [ ]:
from typing import Optional, Union, List

from segmentation_models_pytorch.encoders import get_encoder
from segmentation_models_pytorch.base import (
    SegmentationModel,
    SegmentationHead,
    ClassificationHead,
)

class FPANet(SegmentationModel):
    def __init__(
        self,
        encoder_name: str = "resnet34",
        encoder_depth: int = 5,
        encoder_weights: Optional[str] = "imagenet",
        decoder_use_batchnorm: bool = True,
        decoder_channels: List[int] = (256, 128, 64, 32, 16),
        decoder_pab_channels: int = 64,
        in_channels: int = 3,
        classes: int = 1,
        activation: Optional[Union[str, callable]] = None,
        aux_params: Optional[dict] = None,
    ):
        super().__init__()

        self.encoder = get_encoder(
            encoder_name,
            in_channels=in_channels,
            depth=encoder_depth,
            weights=encoder_weights,
        )

        self.decoder = FPAnetDecoder(
            encoder_channels=self.encoder.out_channels,
            decoder_channels=decoder_channels,
            n_blocks=encoder_depth,
            use_batchnorm=decoder_use_batchnorm,
            pab_channels=decoder_pab_channels,
        )

        self.segmentation_head = SegmentationHead(
            in_channels=decoder_channels[-1],
            out_channels=classes,
            activation=activation,
            kernel_size=3,
        )

        if aux_params is not None:
            self.classification_head = ClassificationHead(in_channels=self.encoder.out_channels[-1], **aux_params)
        else:
            self.classification_head = None

        self.name = "manet-{}".format(encoder_name)
        self.initialize()

Build Loss and Model (FPANet)

In [ ]:
from segmentation_models_pytorch.losses import DiceLoss
from segmentation_models_pytorch.metrics import get_stats, iou_score, accuracy, precision, recall, f1_score

class SegmentationModelFPANet(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = FPANet(
            encoder_name="resnet34",
            encoder_weights="imagenet",
            decoder_channels=(256, 128, 64, 32, 16),
            in_channels=3,
            classes=len(CLASSES),
            activation="softmax"
        )
        self.FreqFeature = FrequencyFeature()
        self.criterion = DiceLoss(mode="multiclass", from_logits=False)

    def forward(self, inputs, targets=None):
        inputs = self.FreqFeature(inputs)
        outputs = self.model(inputs)
        if targets is not None:
            loss = self.criterion(outputs, targets)
            tp, fp, fn, tn = get_stats(outputs.argmax(dim=1).unsqueeze(1).type(torch.int64), targets, mode='multiclass', num_classes=len(CLASSES))
            metrics = {
                "Accuracy": accuracy(tp, fp, fn, tn, reduction="micro-imagewise"),
                "IoU": iou_score(tp, fp, fn, tn, reduction="micro-imagewise"),
                "Precision": precision(tp, fp, fn, tn, reduction="micro-imagewise"),
                "Recall": recall(tp, fp, fn, tn, reduction="micro-imagewise"),
                "F1score": f1_score(tp, fp, fn, tn, reduction="micro-imagewise")
            }
            return loss, metrics, outputs
        else:
            return outputs

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=0.0001)

    def training_step(self, batch, batch_idx):
        images, masks = batch

        loss, metrics, outputs = self(images, masks)
        self.log_dict({
            "train/Loss": loss,
            "train/IoU": metrics['IoU'],
            "train/Accuracy": metrics['Accuracy'],
            "train/Precision": metrics['Precision'],
            "train/Recall": metrics['Recall'],
            "train/F1score": metrics['F1score']
        }, prog_bar=True, logger=True, on_step=False, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        images, masks = batch

        loss, metrics, outputs = self(images, masks)
        self.log_dict({
            "val/Loss": loss,
            "val/IoU": metrics['IoU'],
            "val/Accuracy": metrics['Accuracy'],
            "val/Precision": metrics['Precision'],
            "val/Recall": metrics['Recall'],
            "val/F1score": metrics['F1score']
        }, prog_bar=True, logger=True, on_step=False, on_epoch=True)
        return loss

    def test_step(self, batch, batch_idx):
        images, masks = batch

        loss, metrics, outputs = self(images, masks)
        self.log_dict({
            "test/Loss": loss,
            "test/IoU": metrics['IoU'],
            "test/Accuracy": metrics['Accuracy'],
            "test/Precision": metrics['Precision'],
            "test/Recall": metrics['Recall'],
            "test/F1score": metrics['F1score']
        }, prog_bar=True, logger=True, on_step=False, on_epoch=True)
        return loss

In [ ]:
from torchinfo import summary

model = SegmentationModelFPANet()
summary(model, input_size=(BATCH_SIZE, 3, IMAGE_SIZE, IMAGE_SIZE))

Train the Model

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import CSVLogger
import math

checkpoint_callback = ModelCheckpoint(
    dirpath="drive/MyDrive/MLProject/LandCover/DeepGlobe/checkpoints",
    filename="best-checkpoint-FPANet",
    save_top_k=1,
    verbose=True,
    monitor="val/Loss",
    mode="min"
)

logger = CSVLogger("drive/MyDrive/MLProject/LandCover/DeepGlobe/lightning_logs", name="landcover-log-FPANet")

early_stopping_callback = EarlyStopping(monitor="val/Loss", patience=10)
pl.seed_everything(42, )
trainer = pl.Trainer(
    logger=logger,
    log_every_n_steps=math.ceil(len(pd_train)/BATCH_SIZE),
    callbacks=[checkpoint_callback, early_stopping_callback],
    max_epochs=EPOCHS,
    accelerator="auto",
    devices=1
)

### Start Training (DeepGlobe Dataset)

In [ ]:
trainer.fit(model, data_module.train_dataloader(), data_module.val_dataloader())

In [ ]:
metrics = pd.read_csv("drive/MyDrive/MLProject/LandCover/DeepGlobe/lightning_logs/landcover-log-FPANet/version_0/metrics.csv")
fig, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(2, 3, figsize=(18, 10))

axes = [ax1, ax2, ax3, ax4, ax5, ax6]
names = ['Loss', 'IoU', 'Accuracy', 'Precision', 'Recall', 'F1score']

for axis, name in zip(axes, names):
    epochs = list(range(len(metrics[f'train/{name}'].dropna())))
    axis.plot(epochs, metrics[f'train/{name}'].dropna())
    axis.plot(epochs, metrics[f'val/{name}'].dropna())
    axis.set_title(f'{name}: Train/Val')
    axis.set_ylabel(name)
    axis.set_xlabel('Epoch')
    #axis.set_xlim(0,5)
    ax1.legend(['training', 'validation'])

### Start Testing

Test the Model

In [ ]:
trainer.test(model, ckpt_path="drive/MyDrive/MLProject/LandCover/DeepGlobe/checkpoints/best-checkpoint-FPANet.ckpt", dataloaders = data_module.test_dataloader())

### Evaluate the model with single image from test dataset

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
model = SegmentationModelFPANet.load_from_checkpoint("drive/MyDrive/MLProject/LandCover/DeepGlobe/checkpoints/best-checkpoint-FPANet.ckpt", map_location=torch.device(DEVICE))
model.eval()

In [ ]:
image, mask = next(iter(data_module.test_dataloader()))
image.shape, mask.shape


In [ ]:
img1 = image[7,:,:,:]
mask1 = mask[7,:,:,:]

In [ ]:
img1 = torch.unsqueeze(img1, axis=0)
mask1 = torch.unsqueeze(mask1, axis=0)

In [ ]:
img1.shape, mask1.shape

In [ ]:
with torch.no_grad():
    y_hat = model.to(DEVICE)(img1.to(DEVICE))

In [ ]:
y_hat_1c = torch.argmax(y_hat, dim=1,keepdim=True)

In [ ]:
y_hat_1c.shape

In [ ]:
y_hat_1c_sqz = torch.squeeze(y_hat_1c)

In [ ]:
y_hat_1c_sqz.shape

In [ ]:
if DEVICE == "cuda":
    y_hat_1c_sqz = y_hat_1c_sqz.to("cpu")
y_hat_final = category2rgb(y_hat_1c_sqz)

In [ ]:
y_hat_final.shape


In [ ]:
mask1.shape

In [ ]:
mask1_final = category2rgb(torch.squeeze(mask1))

In [ ]:
mask1_final.shape

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 10))

ax1.set_title('Original mask')
ax1.imshow(mask1_final)

ax2.set_title('Prediction')
ax2.imshow(y_hat_final)

### Evaluate the model with a random image

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
model = SegmentationModelFPANet.load_from_checkpoint("drive/MyDrive/MLProject/LandCover/DeepGlobe/checkpoints/best-checkpoint-FPANet.ckpt", map_location=torch.device(DEVICE))
model.eval()
index = 190

In [ ]:

img_dim = (IMAGE_SIZE, IMAGE_SIZE)
img1 = cv2.imread(pd_test.iloc[index].IMAGES)
img1 = cv2.resize(img1,img_dim)
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)

mask1 = cv2.imread(pd_test.iloc[index].MASKS)
mask1 = cv2.resize(mask1,img_dim)
mask1 = cv2.cvtColor(mask1, cv2.COLOR_BGR2RGB)

In [ ]:
img1.shape, mask1.shape

In [ ]:
mask1 = rgb2category(mask1)

img1 = np.transpose(img1, (2, 0, 1)).astype(np.float64)
mask1 = np.expand_dims(mask1, axis=0)

img1 = torch.Tensor(img1) / 255.0
mask1 = torch.Tensor(mask1).long()

In [ ]:
img1.shape, mask1.shape

In [ ]:
img1 = torch.unsqueeze(img1, axis=0)
mask1 = torch.unsqueeze(mask1, axis=0)

In [ ]:
img1.shape, mask1.shape

In [ ]:
with torch.no_grad():
    y_hat = model.to(DEVICE)(img1.to(DEVICE))

In [ ]:
y_hat.shape

In [ ]:
y_hat_1c = torch.argmax(y_hat, dim=1,keepdim=True)

In [ ]:
y_hat_1c.shape

In [ ]:
y_hat_1c_sqz = torch.squeeze(y_hat_1c)

In [ ]:
y_hat_1c_sqz.shape

In [ ]:
if DEVICE == "cuda":
    y_hat_1c_sqz = y_hat_1c_sqz.to("cpu")
y_hat_final = category2rgb(y_hat_1c_sqz)

In [ ]:
y_hat_final.shape


In [ ]:
mask1.shape

In [ ]:
mask1_final = category2rgb(torch.squeeze(mask1))

In [ ]:
mask1_final.shape

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 10))

ax1.set_title('Original mask')
ax1.imshow(mask1_final)

ax2.set_title('Prediction')
ax2.imshow(y_hat_final)

### Save Figures

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model = SegmentationModelFPANet.load_from_checkpoint("drive/MyDrive/MLProject/LandCover/DeepGlobe/checkpoints/best-checkpoint-FPANet.ckpt", map_location=torch.device(DEVICE))
model.eval()
test_dataset_len = len(pd_test)
print(f'Test dataset length: {test_dataset_len}\n')
for index in range(test_dataset_len):
  img_dim = (IMAGE_SIZE, IMAGE_SIZE)
  img1 = cv2.imread(pd_test.iloc[index].IMAGES)
  img1 = cv2.resize(img1,img_dim)
  img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
  #new
  org_img = img1

  mask1 = cv2.imread(pd_test.iloc[index].MASKS)
  mask1 = cv2.resize(mask1,img_dim)
  mask1 = cv2.cvtColor(mask1, cv2.COLOR_BGR2RGB)

  mask1 = rgb2category(mask1)

  img1 = np.transpose(img1, (2, 0, 1)).astype(np.float64)
  mask1 = np.expand_dims(mask1, axis=0)

  img1 = torch.Tensor(img1) / 255.0
  mask1 = torch.Tensor(mask1).long()

  img1 = torch.unsqueeze(img1, axis=0)
  mask1 = torch.unsqueeze(mask1, axis=0)

  with torch.no_grad():
    y_hat = model.to(DEVICE)(img1.to(DEVICE))

  y_hat_1c = torch.argmax(y_hat, dim=1,keepdim=True)
  y_hat_1c_sqz = torch.squeeze(y_hat_1c)

  if DEVICE == "cuda":
    y_hat_1c_sqz = y_hat_1c_sqz.to("cpu")
  y_hat_final = category2rgb(y_hat_1c_sqz)

  mask1_final = category2rgb(torch.squeeze(mask1))
  fig, (ax0, ax1, ax2) = plt.subplots(1, 3, figsize=(16, 10))

  ax0.set_title('image')
  ax0.imshow(org_img)
  ax0.axis('off')

  ax1.set_title('Original mask')
  ax1.imshow(mask1_final)
  ax1.axis('off')

  ax2.set_title('Prediction')
  ax2.imshow(y_hat_final)
  ax2.axis('off')
  dir_path = "drive/MyDrive/MLProject/LandCover/DeepGlobe/Output_Figures/FPANet"
  if not os.path.exists(dir_path):
    os.makedirs(dir_path)
  file_name = str(dir_path)+"/FPANet_" + str(index+1) + ".png"
  plt.savefig(file_name)
  print("Saved Figure: ",index+1)
  plt.close()
